# Analyzing Used Car Listings on eBay Kleinanzeigen

Dataset was scraped and retrieved from [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data)

The data dictionary provided with data is as follows:

-  dateCrawled - When this ad was first crawled. All field-values are taken from this date.
-  name - Name of the car.
-  seller - Whether the seller is private or a dealer.
-  offerType - The type of listing
-  price - The price on the ad to sell the car.
-  abtest - Whether the listing is included in an A/B test.
-  vehicleType - The vehicle Type.
-  yearOfRegistration - The year in which which year the car was first registered.
-  gearbox - The transmission type.
-  powerPS - The power of the car in PS.
-  model - The car model name.
-  kilometer - How many kilometers the car has driven.
-  monthOfRegistration - The month in which which year the car was first registered.
-  fuelType - What type of fuel the car uses.
-  brand - The brand of the car.
-  notRepairedDamage - If the car has a damage which is not yet repaired.
-  dateCreated - The date on which the eBay listing was created.
-  nrOfPictures - The number of pictures in the ad.
-  postalCode - The postal code for the location of the vehicle.
-  lastSeenOnline - When the crawler saw this ad last online.


The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding = "Latin-1")

print(autos.isnull().sum())
print("\n")
print(autos.info())
print("\n")
print(autos.columns)
print("\n")

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType        

In [2]:
col_mapping = {"dateCrawled" : "date_crawled", "offerType" : "offer_type", "abtest" : "ab_test", "vehicleType" : "vehicle_type",
              "yearOfRegistration" : "registration_year", "powerPS" : "power_ps", "monthOfRegistration" : "registration_month",
              "fuelType" : "fuel_type", "notRepairedDamage" : "unrepaired_damage", "dateCreated" : "ad_created", "nrOfPictures" : "num_photos",
              "postalCode" : "post_code", "lastSeen" : "last_seen"}

autos = autos.rename(col_mapping, axis = 1)
print(autos.head())
print("\n")

          date_crawled                            name  seller offer_type  \
0  2016-03-24 11:52:17                      Golf_3_1.6  privat    Angebot   
1  2016-03-24 10:58:45            A5_Sportback_2.7_Tdi  privat    Angebot   
2  2016-03-14 12:52:21  Jeep_Grand_Cherokee_"Overland"  privat    Angebot   
3  2016-03-17 16:54:04              GOLF_4_1_4__3TÜRER  privat    Angebot   
4  2016-03-31 17:25:20  Skoda_Fabia_1.4_TDI_PD_Classic  privat    Angebot   

   price ab_test vehicle_type  registration_year    gearbox  power_ps  model  \
0    480    test          NaN               1993    manuell         0   golf   
1  18300    test        coupe               2011    manuell       190    NaN   
2   9800    test          suv               2004  automatik       163  grand   
3   1500    test   kleinwagen               2001    manuell        75   golf   
4   3600    test   kleinwagen               2008    manuell        69  fabia   

   kilometer  registration_month fuel_type       brand u

## Initial Data Cleaning

In [3]:
autos.describe(include = "all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,post_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:45:59
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


In [4]:
autos["num_photos"].value_counts()


0    371528
Name: num_photos, dtype: int64

The values in the num_photos columns seems to be filled with zeroes. Also, the seller and offer_type columns mostly have the same values. As such, I will be removing these columns.

In [5]:
columns_to_drop = ["seller", "offer_type", "num_photos"]
autos.drop(columns_to_drop, axis = 1)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,post_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04 00:00:00,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,Peugeot_206_CC_110_Platinum,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein,2016-04-01 00:00:00,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,VW_Derby_Bj_80__Scheunenfund,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein,2016-03-21 00:00:00,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,Ford_C___Max_Titanium_1_0_L_EcoBoost,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN,2016-04-04 00:00:00,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,VW_Golf_4_5_tuerig_zu_verkaufen_mit_Anhaengerk...,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,27472,2016-03-31 17:17:06


## Translating German Words

There's a few column that contains german words, particularly in the `vehicle_type`, `gearbox`, `unrepaired_damage` and `fuel type`. I would like to translate them into english. 

In [6]:
print(autos['vehicle_type'].value_counts(dropna = False))
print(autos['gearbox'].value_counts(dropna = False))
print(autos['unrepaired_damage'].value_counts(dropna = False))
print(autos['fuel_type'].value_counts(dropna = False))

limousine     95894
kleinwagen    80023
kombi         67564
NaN           37869
bus           30201
cabrio        22898
coupe         19015
suv           14707
andere         3357
Name: vehicle_type, dtype: int64
manuell      274214
automatik     77105
NaN           20209
Name: gearbox, dtype: int64
nein    263182
NaN      72060
ja       36286
Name: unrepaired_damage, dtype: int64
benzin     223857
diesel     107746
NaN         33386
lpg          5378
cng           571
hybrid        278
andere        208
elektro       104
Name: fuel_type, dtype: int64


In [7]:
print(autos.head())
mapping_fix = {'bus':'bus', 'limousine':'limousine', 'kleinwagen':'supermini', 'kombi':'station_wagon', 'coupe':'coupe',
    'suv':'suv', 'cabrio':'cabrio', 'andere' :'other'}
autos['vehicle_type'] = autos['vehicle_type'].map(mapping_fix, na_action = 'ignore')

mapping_fix = {'manuell' : 'manual', 'automatik' : 'automatic'}
autos['gearbox'] = autos['gearbox'].map(mapping_fix, na_action = 'ignore')

mapping_fix = {'nein' : 'no', 'ja' : 'yes'}
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(mapping_fix, na_action = 'ignore')

mapping_fix = {'benzin' : 'petrol', 'diesel' : 'diesel', 'lpg' : 'lpg', 'cng' : 'cng', 'hybrid' : 'hybrid', 'andere' : 'other',
              'elektro' : 'electric'}
autos['fuel_type'] = autos['fuel_type'].map(mapping_fix, na_action = 'ignore')
print(autos.head())

          date_crawled                            name  seller offer_type  \
0  2016-03-24 11:52:17                      Golf_3_1.6  privat    Angebot   
1  2016-03-24 10:58:45            A5_Sportback_2.7_Tdi  privat    Angebot   
2  2016-03-14 12:52:21  Jeep_Grand_Cherokee_"Overland"  privat    Angebot   
3  2016-03-17 16:54:04              GOLF_4_1_4__3TÜRER  privat    Angebot   
4  2016-03-31 17:25:20  Skoda_Fabia_1.4_TDI_PD_Classic  privat    Angebot   

   price ab_test vehicle_type  registration_year    gearbox  power_ps  model  \
0    480    test          NaN               1993    manuell         0   golf   
1  18300    test        coupe               2011    manuell       190    NaN   
2   9800    test          suv               2004  automatik       163  grand   
3   1500    test   kleinwagen               2001    manuell        75   golf   
4   3600    test   kleinwagen               2008    manuell        69  fabia   

   kilometer  registration_month fuel_type       brand u

## Exploring Kilometer and Price

In [8]:
autos["kilometer"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: kilometer, dtype: int64

We can see that the values in this field are rounded. Which might indicate that the sellers had to choose from a pre-set options for this field. There are also seems to be more car with high mileage than the low ones.

In [9]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(20)

(5597,)
count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
999      3364
750      3203
650      3150
4500     3053
850      2946
2200     2936
700      2936
1800     2886
900      2874
950      2793
Name: price, dtype: int64

The prices in this column seem rounded, however given there are 5597 unique values in the column, that may just be people's tendency to round prices on the site.

There are 10778 cars listed with $0 price - given that this around 35\% of the of the cars, we have to reconsider if we want to remove these rows. The maximum price is one hundred million dollars, which seems a lot, let's look at the highest prices further.

In [10]:
autos["price"].value_counts().sort_index(ascending=False).head(20)


2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
Name: price, dtype: int64

In [11]:
autos["price"].value_counts().sort_index(ascending=True).head(20)


0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
Name: price, dtype: int64

There are a number of listings with prices below 20USD, including about 1,100 at 1USD. There are also a small number of listings with very high values, including 15 at around 1 million USD.

Given that eBay is an auction site, there could legitimately be items where the opening bid is. We will keep the items, but remove anything above , since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [12]:
price_filter = (autos["price"] > 0) & (autos["price"] < 350000)
autos = autos[price_filter]
autos["price"].describe()

count    360631.000000
mean       5894.855298
std        8792.034481
min           1.000000
25%        1250.000000
50%        3000.000000
75%        7490.000000
max      349000.000000
Name: price, dtype: float64

## Exploring the Date Column

There are a number of columns with date information:

-  date_crawled
-  registration_month
-  registration_year
-  ad_created
-  last_seen
These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration dates are stored as strings.

We'll explore each of these columns to learn more about the listings.

In [13]:
autos[['date_crawled','ad_created','last_seen']][0:5]


,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [14]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.025547
2016-03-06    0.014483
2016-03-07    0.035657
2016-03-08    0.033469
2016-03-09    0.034115
2016-03-10    0.032646
2016-03-11    0.032773
2016-03-12    0.036242
2016-03-13    0.015783
2016-03-14    0.036328
2016-03-15    0.033425
2016-03-16    0.030205
2016-03-17    0.031647
2016-03-18    0.013119
2016-03-19    0.035272
2016-03-20    0.036400
2016-03-21    0.035679
2016-03-22    0.032493
2016-03-23    0.032002
2016-03-24    0.029914
2016-03-25    0.032801
2016-03-26    0.031975
2016-03-27    0.030225
2016-03-28    0.035064
2016-03-29    0.034126
2016-03-30    0.033536
2016-03-31    0.031872
2016-04-01    0.034146
2016-04-02    0.035094
2016-04-03    0.038813
2016-04-04    0.037626
2016-04-05    0.012780
2016-04-06    0.003128
2016-04-07    0.001617
Name: date_crawled, dtype: float64

In [15]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-04-07    0.001617
2016-04-06    0.003128
2016-04-05    0.012780
2016-03-18    0.013119
2016-03-06    0.014483
2016-03-13    0.015783
2016-03-05    0.025547
2016-03-24    0.029914
2016-03-16    0.030205
2016-03-27    0.030225
2016-03-17    0.031647
2016-03-31    0.031872
2016-03-26    0.031975
2016-03-23    0.032002
2016-03-22    0.032493
2016-03-10    0.032646
2016-03-11    0.032773
2016-03-25    0.032801
2016-03-15    0.033425
2016-03-08    0.033469
2016-03-30    0.033536
2016-03-09    0.034115
2016-03-29    0.034126
2016-04-01    0.034146
2016-03-28    0.035064
2016-04-02    0.035094
2016-03-19    0.035272
2016-03-07    0.035657
2016-03-21    0.035679
2016-03-12    0.036242
2016-03-14    0.036328
2016-03-20    0.036400
2016-04-04    0.037626
2016-04-03    0.038813
Name: date_crawled, dtype: float64

In [16]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.001264
2016-03-06    0.004098
2016-03-07    0.005202
2016-03-08    0.007939
2016-03-09    0.009824
2016-03-10    0.011460
2016-03-11    0.012955
2016-03-12    0.023240
2016-03-13    0.008410
2016-03-14    0.012176
2016-03-15    0.016321
2016-03-16    0.016418
2016-03-17    0.028700
2016-03-18    0.006888
2016-03-19    0.016330
2016-03-20    0.019885
2016-03-21    0.020026
2016-03-22    0.020506
2016-03-23    0.018016
2016-03-24    0.019164
2016-03-25    0.019000
2016-03-26    0.015958
2016-03-27    0.016718
2016-03-28    0.022189
2016-03-29    0.023284
2016-03-30    0.023725
2016-03-31    0.024244
2016-04-01    0.023897
2016-04-02    0.024967
2016-04-03    0.025308
2016-04-04    0.025533
2016-04-05    0.126964
2016-04-06    0.218952
2016-04-07    0.130438
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

## Dealing with Incorrect Registration Year Data

Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:


In [17]:
fltr = (autos["registration_year"] > 1900) & (autos["registration_year"] < 2017)
correct = autos[fltr].shape[0]
correct_prcnt = correct / autos.shape[0]
print(1 - correct_prcnt)


0.03875152163846152


Given that this is less than 4% of our data, we will remove these rows.

In [18]:
clean_autos = autos[fltr]
clean_autos["registration_year"].value_counts(normalize=True).head(10)


2000    0.066700
1999    0.063553
2005    0.062670
2006    0.057709
2001    0.056956
2003    0.056558
2004    0.056174
2002    0.054290
2007    0.050500
1998    0.049692
Name: registration_year, dtype: float64

It looks like most cars sold here were registered in the past 20 years.

## Exploring Price by Brand

In [19]:
brand_counts = clean_autos["brand"].value_counts(normalize = True)
brand_counts

volkswagen        0.211703
bmw               0.109873
opel              0.106411
mercedes_benz     0.096840
audi              0.089544
ford              0.068918
renault           0.047517
peugeot           0.030154
fiat              0.025691
seat              0.018661
skoda             0.015687
mazda             0.015384
smart             0.014331
citroen           0.013950
nissan            0.013598
toyota            0.012932
hyundai           0.009972
sonstige_autos    0.009491
mini              0.009384
volvo             0.009147
mitsubishi        0.008236
honda             0.007532
kia               0.006915
suzuki            0.006364
alfa_romeo        0.006309
porsche           0.006205
chevrolet         0.005022
chrysler          0.003863
dacia             0.002495
jeep              0.002192
land_rover        0.002166
daihatsu          0.002161
subaru            0.002117
jaguar            0.001734
saab              0.001465
daewoo            0.001457
trabant           0.001408
l

German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [20]:
fltr = brand_counts > 0.05
common_brands = brand_counts[fltr]
common_brands

volkswagen       0.211703
bmw              0.109873
opel             0.106411
mercedes_benz    0.096840
audi             0.089544
ford             0.068918
Name: brand, dtype: float64

In [21]:
common_brands_index = common_brands.index

mean_price_per_brand = {}

for brand in common_brands_index:
    fltr = clean_autos["brand"] == brand
    cars = clean_autos[fltr]
    brand_mean = cars["price"].mean()
    mean_price_per_brand[brand] = int(brand_mean)
    
mean_price_per_brand

{'volkswagen': 5400,
 'bmw': 8449,
 'opel': 2971,
 'mercedes_benz': 8541,
 'audi': 9086,
 'ford': 3696}

Of the top 5 brands, there is a distinct price gap:

-  Audi, BMW and Mercedes Benz are more expensive
-  Ford and Opel are less expensive
-  Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.

## Exploring Mileage

In [22]:
mean_mileage_per_brand = {}

for brand in common_brands_index:
    fltr = clean_autos["brand"] == brand
    cars = clean_autos[fltr]
    mile_mean = cars["kilometer"].mean()
    mean_mileage_per_brand[brand] = int(mile_mean)
    
mean_mileage = pd.Series(mean_mileage_per_brand).sort_values(ascending=False)
print(mean_mileage)
print()
mean_prices = pd.Series(mean_price_per_brand).sort_values(ascending=False)
print(mean_prices)

bmw              132800
mercedes_benz    130573
audi             129443
opel             128722
volkswagen       128386
ford             123662
dtype: int64

audi             9086
mercedes_benz    8541
bmw              8449
volkswagen       5400
ford             3696
opel             2971
dtype: int64


In [23]:
brand_info = pd.DataFrame(columns = ["mean_mileage", "mean_prices"])
brand_info

,mean_mileage,mean_prices


In [24]:
brand_info["mean_prices"] = mean_prices
brand_info["mean_mileage"] = mean_mileage

brand_info.sort_values(by = "mean_prices")

,mean_mileage,mean_prices
opel,128722,2971
ford,123662,3696
volkswagen,128386,5400
bmw,132800,8449
mercedes_benz,130573,8541
audi,129443,9086


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.

## Summary

The higher the price does not neccessarily mean better mileage. We can see that Opel is one of the top car brands being sold in the website, and it has `2971` average price with `128722` kilo mileage. Compare it with other non luxury brands like ford and volkswagen with higher mean prices `3696` and `5400` respectively, both brands dont have a much better average mileage than Opel.